<a href="https://colab.research.google.com/github/srdg/unarchived_ben_tess/blob/main/Bengali_Typeset_With_Tesseract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation
Mount tesseract and set up the environment by installing  necessary libraries and packages

In [ ]:
!sudo apt install -q tesseract-ocr

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 14 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 3s (1,609 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/Fron

In [ ]:
!sudo apt install libtesseract-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libleptonica-dev
The following NEW packages will be installed:
  libleptonica-dev libtesseract-dev
0 upgraded, 2 newly installed, 0 to remove and 14 not upgraded.
Need to get 2,755 kB of archives.
After this operation, 13.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libleptonica-dev amd64 1.75.3-3 [1,308 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libtesseract-dev amd64 4.00~git2288-10f4998a-2 [1,447 kB]
Fetched 2,755 kB in 3s (1,010 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This fr

In [ ]:
!which tesseract

/usr/bin/tesseract


In [ ]:
!sudo apt-get install tesseract-ocr-ben

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tesseract-ocr-ben
0 upgraded, 1 newly installed, 0 to remove and 14 not upgraded.
Need to get 515 kB of archives.
After this operation, 870 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-ben all 4.00~git24-0e00fe6-1.2 [515 kB]
Fetched 515 kB in 2s (297 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package tesseract-ocr-ben.
(Reading database .

In [ ]:
!ls /usr/share/tesseract-ocr/4.00/tessdata

ben.traineddata  eng.traineddata  pdf.ttf
configs		 osd.traineddata  tessconfigs


# Downloading data

In [ ]:
!rm -rf ./sample_data # remove default sample data
!rm -rf 010_LetterPress-Data line_hocr.sh # delete old versions of data
!wget -q https://api.github.com/repos/srdg/unarchived_ben_tess/zipball  
!unzip -q zipball && unzip -q ./srdg-*/010_LetterPress.zip
!mv ./srdg-*/line_hocr.sh . #unzip data and take the shell script
!mv ./srdg-*/run.py ./010_LetterPress-Data #unzip data and take py script
!chmod +xxx ./line_hocr.sh # make executable
!rm -rf zipball srdg-* # delete redundant archives

In [ ]:
!wget https://github.com/ocropus/hocr-tools/archive/master.zip && \
unzip -q ./master.zip
!rm -rf ./master.zip # get hocr-tools

--2020-12-13 06:28:28--  https://github.com/ocropus/hocr-tools/archive/master.zip
Resolving github.com (github.com)... 13.114.40.48
Connecting to github.com (github.com)|13.114.40.48|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/ocropus/hocr-tools/zip/master [following]
--2020-12-13 06:28:29--  https://codeload.github.com/ocropus/hocr-tools/zip/master
Resolving codeload.github.com (codeload.github.com)... 13.112.159.149
Connecting to codeload.github.com (codeload.github.com)|13.112.159.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [  <=>               ]   1.46M  4.32MB/s    in 0.3s    

2020-12-13 06:28:29 (4.32 MB/s) - ‘master.zip’ saved [1532153]



# Preprocessing data

In [ ]:
%cd ./010_LetterPress-Data/
!python ./run.py # run python script
%cd .. # at this point, all ground truth text files are generated

In [ ]:
# Run shell script to split image at line level
!bash line_hocr.sh&>./preprocessing.log
# Remove generated gt.txt files
!rm -rf ./010_LetterPress-Data/*.exp0.txt

In [ ]:
# Move the split images into separate folder to correct ground truth
!mkdir -p ./data/ben-ground-truth
!mv ./010_LetterPress-Data/*.exp0.* ./data/ben-ground-truth/

# Training

In [ ]:
!rm -rf tesstrain-master # remove old downloads for session continuity
!wget https://github.com/tesseract-ocr/tesstrain/archive/master.zip && \
unzip -q ./master.zip # unzip training diretory
!rm -rf ./master.zip # remove redundant archive

--2020-12-13 06:34:02--  https://github.com/tesseract-ocr/tesstrain/archive/master.zip
Resolving github.com (github.com)... 13.114.40.48
Connecting to github.com (github.com)|13.114.40.48|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/tesseract-ocr/tesstrain/zip/master [following]
--2020-12-13 06:34:03--  https://codeload.github.com/tesseract-ocr/tesstrain/zip/master
Resolving codeload.github.com (codeload.github.com)... 52.193.111.178
Connecting to codeload.github.com (codeload.github.com)|52.193.111.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [    <=>             ]   5.19M  7.72MB/s    in 0.7s    

2020-12-13 06:34:04 (7.72 MB/s) - ‘master.zip’ saved [5445507]



In [ ]:
# move data to required training location 
# change present working directory
!mv ./data ./tesstrain-master/
%cd ./tesstrain-master/

In [ ]:
# Cleanup
!rm -rf ./data/ben 
!rm -rf radical-stroke.txt
!rm -rf ./data/ben-ground-truth/*.box ./data/ben-ground-truth/*.lstmf

# Install bc
!sudo apt install bc

# Actual training
!make training MODEL_NAME=ben

Streaming output truncated to the last 5000 lines.
Warning. Invalid resolution 0 dpi. Using 70 instead.
PYTHONIOENCODING=utf-8 python3 generate_line_box.py -i "data/ben-ground-truth/31-012.exp0.tif" -t "data/ben-ground-truth/31-012.exp0.gt.txt" > "data/ben-ground-truth/31-012.exp0.box"
+ tesseract data/ben-ground-truth/31-012.exp0.tif data/ben-ground-truth/31-012.exp0 --psm 13 lstm.train
Tesseract Open Source OCR Engine v4.0.0-beta.1 with Leptonica
Page 1
Warning. Invalid resolution 0 dpi. Using 70 instead.
PYTHONIOENCODING=utf-8 python3 generate_line_box.py -i "data/ben-ground-truth/36-009.exp0.tif" -t "data/ben-ground-truth/36-009.exp0.gt.txt" > "data/ben-ground-truth/36-009.exp0.box"
+ tesseract data/ben-ground-truth/36-009.exp0.tif data/ben-ground-truth/36-009.exp0 --psm 13 lstm.train
Tesseract Open Source OCR Engine v4.0.0-beta.1 with Leptonica
Page 1
Warning. Invalid resolution 0 dpi. Using 70 instead.
PYTHONIOENCODING=utf-8 python3 generate_line_box.py -i "data/ben-ground-truth/